In [112]:
import os
import requests
import polars as pl
from statistics import mean
from dotenv import load_dotenv
from datetime import datetime, timezone, timedelta

# Load environment variables from .env file
load_dotenv(".env")

True

In [62]:
def unix_time(date: str) -> float:
    """Take in date in string format and return UNIX timestamp float"""
    stamp = datetime.strptime(date, '%Y/%m/%d %H:%M:%S')
    return int(stamp.replace(tzinfo=timezone.utc).timestamp())

In [119]:
def dates_between(start: str, end: str) -> list:
    """Return a list of first dates of each year, given a start and end"""
    start_date = datetime.strptime(start, '%Y/%m/%d')
    end_date = datetime.strptime(end, '%Y/%m/%d')
    date_list = []

    # Check if the start date is the first day of the year
    if start_date.month == 1 and start_date.day == 1:
        current_date = start_date
    else:
        # If not, use the first day of current year
        current_date = start_date.replace(month=1, day=1)

    while end_date > current_date:
        # Add current date to the list of dates and replace the month and day to 1
        date_list.append(current_date)
        current_date = current_date.replace(month=1, day=1)
        # Add one year from the current date and replace the month and day to 1
        current_date = current_date.replace(year=current_date.year + 1, month=1, day=1)
        
    # Add first day of next year for end date
    date_list.append(end_date.replace(year=end_date.year + 1, month=1, day=1))

    # Return list of dates as strings
    return [date.strftime('%Y/%m/%d') + ' 00:00:00' for date in date_list]

In [124]:
def api_call(dates_list: list) -> dict:
    """Make a call to the Owlracle API and return historical gas JSON"""
    key = os.getenv('OAPI')
    results = {}    # Dict of JSON

    # Loop over the list of dates in pairs using zip
    for start_date, end_date in zip(dates_list, dates_list[1:]):
        unix_start = unix_time(start_date)
        unix_end = unix_time(end_date)
    
        # Make the API call with the start and end dates
        res = requests.get(f'https://api.owlracle.info/v4/eth/history?apikey={key}&from={unix_start}&to={unix_end}&candles=365&timeframe=1d&txfee=true')
        # Add the response JSON to the results dictionary with the year as the key
        year = datetime.strptime(start_date, '%Y/%m/%d %H:%M:%S').year
        results[year] = res.json()
        
    return results

In [184]:
def extract_fees(json_data: dict) -> dict:
    """Extract the txFee from the JSON data, calculate the average gas fees, and return a single dictionary"""
    txFees = {}
    for year, data in json_data.items():
        for candle in data['candles']:
            # Extract the txFee from each candle and calculate the average
            low = candle['txFee']['low']
            high = candle['txFee']['high']
            avg = mean([low, high])
            
            # Only add in dates from that year
            date = candle['timestamp'].split('T')[0]
            if date.startswith(str(year)):
                txFees[date] = avg
        
    # Return combined dict in chronological order
    return dict(sorted(txFees.items(), key=lambda x: x[0]))